In [34]:
import polars as pl
import pandas as pd
from pathlib import Path
from quantbullet.utils.encrypt import decrypt_variable_from_file

## PMMS

In [4]:
pmms_polars = pmms_polars.select( ['asofmon', 'pmms30'] ).with_columns(
    yyyymm_str = ( pl.col( "asofmon" ).dt.strftime( "%Y%m" ) )
).with_columns(
    pmms_lag1 = pl.col( "pmms30" ).shift( 1 ),
    pmms_lag2 = pl.col( "pmms30" ).shift( 2 ),
).with_columns(
    pmms_3m_avg = ( pl.col( "pmms30" ) ).rolling_mean( window_size=3 ),
    pmms_6m_avg = ( pl.col( "pmms30" ) ).rolling_mean( window_size=6 ),
    pmms_1y_avg = ( pl.col( "pmms30" ) ).rolling_mean( window_size=12 ),
    pmms_2y_avg = ( pl.col( "pmms_lag1" ) ).rolling_mean( window_size=24 ),
    pmms_3y_avg = ( pl.col( "pmms_lag1" ) ).rolling_mean( window_size=36 ),
    pmms_lag_2_3m_avg = ( pl.col( "pmms_lag2" ) ).rolling_mean( window_size=3 ),
    pmms_lag_2_6m_avg = ( pl.col( "pmms_lag2" ) ).rolling_mean( window_size=6 ),
    pmms_lag_2_1y_avg = ( pl.col( "pmms_lag2" ) ).rolling_mean( window_size=12 ),
    pmms_lag_2_2y_avg = ( pl.col( "pmms_lag2" ) ).rolling_mean( window_size=24 ),
    pmms_lag_2_3y_avg = ( pl.col( "pmms_lag2" ) ).rolling_mean( window_size=36 ),
).with_columns(
    media_3y_3m = ( pl.col( "pmms_3y_avg" ) / pl.col( "pmms_3m_avg" ) ),
    media_3y_6m = ( pl.col( "pmms_3y_avg" ) / pl.col( "pmms_6m_avg" ) ),
    media_3y_3m_lag2 = ( pl.col( "pmms_lag_2_3y_avg" ) / pl.col( "pmms_lag_2_3m_avg" ) ),
    media_3y_6m_lag2 = ( pl.col( "pmms_lag_2_3y_avg" ) / pl.col( "pmms_lag_2_6m_avg" ) ),
).drop(
    pl.col( "asofmon" )
)

## HPI

In [5]:
hpi_polars = hpi_polars.with_columns(
    asofdate_dt = pl.col( "asofdate" ).cast( pl.Utf8 ).str.strptime( pl.Date, format="%Y%m%d" ),
    yyyymm_str = pl.col( "asofdate" ).cast( pl.Utf8 ).str.slice( 0, 6 )
).rename( { "hpi_index" : "hpi" } )

hpi_polars = hpi_polars.with_columns(
    hpi_lag_2 = pl.col( "hpi" ).shift(2).over( ("geo_type", "geo_code"), order_by="yyyymm_str", descending=False ),
    hpi_lag_26 = pl.col( "hpi" ).shift(26).over( ("geo_type", "geo_code"), order_by="yyyymm_str", descending=False )
)

# hpi_polars_zip5 = hpi_polars.filter( pl.col( "geo_type" ) == "ZIP" ).rename( { "geo_code": "zip5", "hpi_index" : "hpi_zip5" } )
# hpi_polars_zip3 = hpi_polars.filter( pl.col( "geo_type" ) == "ZIP3" ).rename( { "geo_code": "zip3", "hpi_index" : "hpi_zip3" } )
# hpi_polars_state = hpi_polars.filter( pl.col( "geo_type" ) == "STATE" ).rename( { "geo_code": "state", "hpi_index" : "hpi_state" } )

zip5_ref = hpi_polars.filter( pl.col( "geo_type" ) == "ZIP" )[ 'geo_code' ].unique().to_list()
zip3_ref = hpi_polars.filter( pl.col( "geo_type" ) == "ZIP3" )[ 'geo_code' ].unique().to_list()
state_ref = hpi_polars.filter( pl.col( "geo_type" ) == "STATE" )[ 'geo_code' ].unique().to_list()

In [190]:
# hpi_polars.filter( pl.col( "geo_code" ) == "CA" ).sort( "year_month_str" )

## PMMS

## Polars Version

In [26]:
data_polars_dlq = data_polars.with_columns(
  months_to_paid = (
    ( pl.col( "paid_to_date" ).dt.year() - pl.col( "factor_date" ).dt.year() ) * 12
    + ( pl.col( "paid_to_date" ).dt.month() - pl.col( "factor_date" ).dt.month() )
    - 1 ),
  zip5 = ( pl.col( "postal_code" ) ),
  zip3 =  ( pl.col( "postal_code" ).str.slice( 0, 3 ) ),
  yyyymm_str = ( pl.col( "factor_date" ).dt.strftime( "%Y%m" ) ),
  orig_yyyymm_str = ( pl.col( "origination_date" ).dt.strftime( "%Y%m" ) ),
  month_str = ( pl.col( "factor_date" ).dt.strftime( "%b" ) ),
  month_int = ( pl.col( "factor_date" ).dt.month() ),
).with_columns(
  dlq_months = pl.col( "months_to_paid" ).clip( lower_bound = 0 )
).with_columns(
  dlq_status = (
    pl.when( ( pl.col( "mba_dlq_status" ) == "0" ) | ( pl.col( "current_actual_bal" ) <= 0.001 ) )
      .then( pl.lit( "PD" ) )
    .when( pl.col( "mba_dlq_status" ) == "C" )
      .then( pl.lit( "C" ) )
    .when( pl.col( "mba_dlq_status" ) == "3" )
      .then( pl.lit( "M30" ) )
    .when( pl.col( "mba_dlq_status" ) == "6" )
      .then( pl.lit( "M60" ) )
    .when( pl.col( "mba_dlq_status" ) == "F" )
      .then( pl.lit( "FCLS" ) )
    .when( pl.col( "mba_dlq_status" ) == "R" )
      .then( pl.lit( "REO" ) )
    .when( pl.col( "mba_dlq_status" ) == "9" )
      .then(
        pl.when( pl.col( "dlq_months" ) <= 3 ).then( pl.lit( "M90" ) )
        .when( pl.col( "dlq_months" ) == 4 ).then( pl.lit( "M120" ) )
        .when( pl.col( "dlq_months" ) == 5 ).then( pl.lit( "M150" ) )
        .when( pl.col( "dlq_months" ) == 6 ).then( pl.lit( "M180" ) )
        .when( pl.col( "dlq_months" ) == 7 ).then( pl.lit( "M210" ) )
        .when( pl.col( "dlq_months" ) == 8 ).then( pl.lit( "M240" ) )
        .otherwise( pl.lit( "M270P" ) )
      )
    .otherwise( pl.lit( "X" ) )
  )
).with_columns(
    next_status = pl.col( "dlq_status" ).shift( -1 ).over( ( "pool_id", "loan_id" ), order_by="factor_date", descending=False ),
    prev_status = pl.col( "dlq_status" ).shift( 1 ).over( ( "pool_id", "loan_id" ), order_by="factor_date", descending=False ),
    prev_bal = pl.col( "current_actual_bal" ).shift( 1 ).over( ( "pool_id", "loan_id" ), order_by="factor_date", descending=False ),
    ever_m30 = pl.when( pl.col( "dlq_status" ) == "M30" ).then( 1 ) .otherwise( 0 ).cum_max().\
      over( ( "pool_id", "loan_id" ), order_by="factor_date", descending=False ),
    ever_m60 = pl.when( pl.col( "dlq_status" ) == "M60" ).then( 1 ) .otherwise( 0 ).cum_max().\
      over( ( "pool_id", "loan_id" ), order_by="factor_date", descending=False ),
    ever_m90 = pl.when( pl.col( "dlq_status" ) == "M90" ).then( 1 ) .otherwise( 0 ).cum_max().\
      over( ( "pool_id", "loan_id" ), order_by="factor_date", descending=False ),
).with_columns(
    Cto0 = pl.col( "next_status" ).eq( "PD" ) * 1
).remove(
    ( pl.col( "prev_bal" ).is_not_null() ) & ( pl.col( "prev_bal") <= 0.01 )
)

data_polars_hpi = data_polars_dlq.with_columns(
    pl.when( pl.col( "zip5" ).is_in( zip5_ref ) ).then( pl.lit( "ZIP" ) )
    .when( pl.col( "zip3" ).is_in( zip3_ref ) ).then( pl.lit( "ZIP3" ) )
    .when( pl.col( "property_state" ).is_in( state_ref ) ).then( pl.lit( "STATE" ) )
    .otherwise( pl.lit( "X" ) )
    .alias( "geo_type" )
).with_columns(
    pl.when( pl.col( "geo_type" ) == "ZIP" ).then(
        pl.col( "zip5" )
    ).when( pl.col( "geo_type" ) == "ZIP3" ).then(
        pl.col( "zip3" )
    ).when( pl.col( "geo_type" ) == "STATE" ).then(
        pl.col( "property_state" )
    ).otherwise(
        pl.lit( "X" )
    ).alias( "geo_code" )
).join(
    hpi_polars.select( [ "yyyymm_str", "geo_type", "geo_code", "hpi", "hpi_lag_2", "hpi_lag_26" ] ),
    left_on = [ "yyyymm_str", "geo_type", "geo_code" ],
    right_on = [ "yyyymm_str", "geo_type", "geo_code" ],
    how = "left"
).join(
    hpi_polars.select( [ "yyyymm_str", "geo_type", "geo_code", "hpi"] ).rename( { "hpi" : "hpi_orig" } ),
    left_on = [ "orig_yyyymm_str", "geo_type", "geo_code" ],
    right_on = [ "yyyymm_str", "geo_type", "geo_code" ],
    how = "left",
).with_columns(
    hpa_lag_2_24m = ( pl.col( "hpi_lag_2" ) / pl.col( "hpi_lag_26" ) ),
    hpa_ratio_since_orig = pl.col( "hpi" ) / pl.col( "hpi_orig" ),
    orig_house_price = pl.col( "orig_bal" ) / ( pl.col( "orig_ltv" ) / 100 ),
).with_columns(
    # there is a question here to whether to use current_scheduled_bal or current_actual_bal
    cltv = ( pl.col( "current_actual_bal" ) / ( pl.col( "orig_house_price" ) * pl.col( "hpa_ratio_since_orig" ) ) ) * 100
)


data_polars_pmms = data_polars_hpi.join(
    pmms_polars,
    left_on = [ "yyyymm_str" ],
    right_on = [ "yyyymm_str" ],
    how = "left"
).with_columns(
    sato = pl.col( "orig_coupon" ) - pl.col( "pmms30" ),
    orig_incentive_spread = pl.col( "orig_coupon" ) - pl.col( "pmms30" ),
    orig_incentive_spread_lag1 = pl.col( "orig_coupon" ) - pl.col( "pmms_lag1" ),
    orig_incentive_spread_lag2 = pl.col( "orig_coupon" ) - pl.col( "pmms_lag2" ),
    incentive_spread = pl.col( "current_coupon" ) - pl.col( "pmms30" ),
    incentive_spread_lag1 = pl.col( "current_coupon" ) - pl.col( "pmms_lag1" ),
    incentive_spread_lag2 = pl.col( "current_coupon" ) - pl.col( "pmms_lag2" ),
)

data_polar_types = data_polars_pmms.with_columns(
    # case when 1 then 1, else 2 for everthing else
    pl.when( pl.col( "number_of_units" ) == 1 ).then( pl.lit( '1' ) ).otherwise( pl.lit( '2' ) ).alias( "f_n_units" ),
    pl.when( pl.col( "ever_m30" ) == 1 ).then( pl.lit( 'Y' ) ).otherwise( pl.lit( 'N' ) ).alias( "f_ever_m30" ),
    pl.when( pl.col( "ever_m60" ) == 1 ).then( pl.lit( 'Y' ) ).otherwise( pl.lit( 'N' ) ).alias( "f_ever_m60" ),
    pl.when( pl.col( "ever_m90" ) == 1 ).then( pl.lit( 'Y' ) ).otherwise( pl.lit( 'N' ) ).alias( "f_ever_m90" )
).with_columns(
    pl.col( "month_str" ).cast( pl.Categorical ).alias( "f_month" ),
    pl.col( "loan_purpose" ).cast( pl.Categorical ).alias( "f_loan_purpose" ),
    pl.col( "property_type" ).cast( pl.Categorical ).alias( "f_property_type" ),
    pl.col( "occ_type" ).cast( pl.Categorical ).alias( "f_occ_type" ),
    pl.col( "month_int" ).cast( pl.UInt8 ).cast( pl.Utf8 ).alias( "f_month_int" ),
    pl.col( "f_n_units" ).cast( pl.Categorical ).alias( "f_n_units" ),
    pl.col( "f_ever_m30" ).cast( pl.Categorical ).alias( "f_ever_m30" ),
    pl.col( "f_ever_m60" ).cast( pl.Categorical ).alias( "f_ever_m60" ),
    pl.col( "f_ever_m90" ).cast( pl.Categorical ).alias( "f_ever_m90" ),
    pl.col( "first_time_buyer" ).cast( pl.Categorical ).alias( "f_first_time_buyer" )
)

NameError: name 'zip5_ref' is not defined

In [9]:
data_polar_types.write_parquet(
    Path( "H:/Data" ) / "20250915-jumbo-deal-full-data-polars.parquet"
)

In [132]:
# data_polar_types.filter(
#     pl.col( "ever_m30" ).eq( True )
# )

data_polar_types['f_loan_purpose'].value_counts()

f_loan_purpose,count
cat,u32
"""P""",20682
"""N""",9802
"""C""",3579


In [121]:
data_polar_types.filter(
    pl.col( "loan_id" ).eq( "000304026012" )
).sort( "factor_date" ).to_pandas().to_clipboard()

In [63]:
data_polar_types['number_of_units'].value_counts()

number_of_units,count
u8,u32
4,189
3,440
2,604
1,32830


In [52]:
data_polars_pmms.select([pl.col(c).is_null().any().alias(c + "_has_null") for c in data_polars_pmms.columns])

deal_name_has_null,pool_id_has_null,factor_date_has_null,age_has_null,current_coupon_has_null,current_actual_bal_has_null,current_scheduled_bal_has_null,date_referred_to_foreclosure_has_null,paid_to_date_has_null,mba_dlq_status_has_null,loan_id_has_null,product_type_has_null,property_state_has_null,lien_has_null,postal_code_has_null,first_payment_date_has_null,maturity_date_has_null,original_loan_term_has_null,orig_coupon_has_null,close_rate_has_null,orig_bal_has_null,close_bal_has_null,prepayment_penalty_indicator_has_null,fico_has_null,dti_ratio_has_null,pmi_has_null,servicer_has_null,origination_date_has_null,orig_ltv_has_null,orig_comb_ltv_has_null,number_of_units_has_null,loan_purpose_has_null,loan_source_has_null,property_type_has_null,occ_type_has_null,first_time_buyer_has_null,months_to_paid_has_null,…,dlq_status_has_null,next_status_has_null,Cto0_has_null,geo_type_has_null,geo_code_has_null,hpi_has_null,hpi_lag_2_has_null,hpi_lag_26_has_null,hpi_orig_has_null,hpa_lag_2_24m_has_null,hpa_ratio_since_orig_has_null,orig_house_price_has_null,cltv_has_null,pmms30_has_null,pmms_lag1_has_null,pmms_lag2_has_null,pmms_3m_avg_has_null,pmms_6m_avg_has_null,pmms_1y_avg_has_null,pmms_2y_avg_has_null,pmms_3y_avg_has_null,pmms_lag_2_3m_avg_has_null,pmms_lag_2_6m_avg_has_null,pmms_lag_2_1y_avg_has_null,pmms_lag_2_2y_avg_has_null,pmms_lag_2_3y_avg_has_null,media_3y_3m_has_null,media_3y_6m_has_null,media_3y_3m_lag2_has_null,media_3y_6m_lag2_has_null,sato_has_null,orig_incentive_spread_has_null,orig_incentive_spread_lag1_has_null,orig_incentive_spread_lag2_has_null,incentive_spread_has_null,incentive_spread_lag1_has_null,incentive_spread_lag2_has_null
bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,…,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
false,false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,…,false,false,false,false,false,true,true,true,true,true,true,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false


In [53]:
n_rows = data_polars_pmms.height
data_polars_pmms.select([
    (pl.col(c).is_null().sum() / n_rows * 100).alias(c + "_pct_null")
    for c in data_polars_pmms.columns
])


deal_name_pct_null,pool_id_pct_null,factor_date_pct_null,age_pct_null,current_coupon_pct_null,current_actual_bal_pct_null,current_scheduled_bal_pct_null,date_referred_to_foreclosure_pct_null,paid_to_date_pct_null,mba_dlq_status_pct_null,loan_id_pct_null,product_type_pct_null,property_state_pct_null,lien_pct_null,postal_code_pct_null,first_payment_date_pct_null,maturity_date_pct_null,original_loan_term_pct_null,orig_coupon_pct_null,close_rate_pct_null,orig_bal_pct_null,close_bal_pct_null,prepayment_penalty_indicator_pct_null,fico_pct_null,dti_ratio_pct_null,pmi_pct_null,servicer_pct_null,origination_date_pct_null,orig_ltv_pct_null,orig_comb_ltv_pct_null,number_of_units_pct_null,loan_purpose_pct_null,loan_source_pct_null,property_type_pct_null,occ_type_pct_null,first_time_buyer_pct_null,months_to_paid_pct_null,…,dlq_status_pct_null,next_status_pct_null,Cto0_pct_null,geo_type_pct_null,geo_code_pct_null,hpi_pct_null,hpi_lag_2_pct_null,hpi_lag_26_pct_null,hpi_orig_pct_null,hpa_lag_2_24m_pct_null,hpa_ratio_since_orig_pct_null,orig_house_price_pct_null,cltv_pct_null,pmms30_pct_null,pmms_lag1_pct_null,pmms_lag2_pct_null,pmms_3m_avg_pct_null,pmms_6m_avg_pct_null,pmms_1y_avg_pct_null,pmms_2y_avg_pct_null,pmms_3y_avg_pct_null,pmms_lag_2_3m_avg_pct_null,pmms_lag_2_6m_avg_pct_null,pmms_lag_2_1y_avg_pct_null,pmms_lag_2_2y_avg_pct_null,pmms_lag_2_3y_avg_pct_null,media_3y_3m_pct_null,media_3y_6m_pct_null,media_3y_3m_lag2_pct_null,media_3y_6m_lag2_pct_null,sato_pct_null,orig_incentive_spread_pct_null,orig_incentive_spread_lag1_pct_null,orig_incentive_spread_lag2_pct_null,incentive_spread_pct_null,incentive_spread_lag1_pct_null,incentive_spread_lag2_pct_null
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.712298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.642926,0.642926,0.642926,0.642926,0.642926,0.642926,0.0,0.642926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
# The missing HPI values are due to MT state
data_polars_hpi.filter( pl.col( "hpi" ).is_null() )[ [ 'year_month_str', 'property_state' ] ].unique( maintain_order=True )

year_month_str,property_state
str,str
"""202503""","""MT"""
"""202502""","""MT"""
"""202501""","""MT"""
"""202412""","""MT"""
"""202411""","""MT"""
…,…
"""202006""","""MT"""
"""202507""","""MT"""
"""202506""","""MT"""


In [55]:
"MT" in state_ref

False